In [1]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random



parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="cwq", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama-3", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
args = parser.parse_args("")


datas, question_string = prepare_dataset(args.dataset)

In [2]:
data = datas[2]
question = data[question_string]
topics = data['topic_entity']
paths = {topics[topic]: {} for topic in topics}
print(question)

What is the predominant religion where the leader is Ovadia Yosef?


In [3]:
for topic in topics:
    topic_name = topics[topic]
    relations = get_relations(question, topic, topic_name, args, 5)
    entities_id, entities_name = get_entities(topic, relations)
    [paths[topic_name].update({r: {"entities_id": entities_id[i], "entities_name": entities_name[i]}}) for i, r in enumerate(relations)]
    facts = propagate(question, topic_name, paths[topic_name], args)
    [paths[topic_name][r].update({"fact": facts[i]}) for i, r in enumerate(relations)]

In [7]:
for topic in topics:
    topic_name = topics[topic]
    relations = get_relations_2hop(question, topic, topic_name, paths[topic_name], args, 3)
    entities_id, entities_name = get_entities_2hop_(paths[topic_name], relations)
    [paths[topic_name].update({r: {"entities_id": entities_id[i], "entities_name": entities_name[i]}}) for i, r in enumerate(relations)]

    # facts = propagate(question, propagate_list, args)
    # [paths[topic_name][r].update({"fact": facts[i]}) for i, r in enumerate(relations)]

In [ ]:
def get_entities_2hop_(paths, relations):
    ids, names = [], []
    for relation in relations:
        entities_id, entities_name = [], []
        topics = paths[relation.split('->')[-2]]['entities_id']
        for topic in topics:
            tail_entities = execute_sparql(sparql_tail_entities % (topic, relation.split('->')[-1]))
            ### !!! some relations like m.04n32 --> music.artist.track has 8477 tail entities
            tail_entities_id, tail_entities_name = filter_entities(tail_entities[:50], topic) 
            entities_id.append(tail_entities_id)
            entities_name.append(tail_entities_name)
        ids.append(entities_id)
        names.append(entities_name)
    return ids, names

In [6]:
get_entities_2hop_(paths[topic_name], relations)

([[['m.0468s5y']],
  [['m.048bcbz']],
  [['m.02wvcg4', 'm.05px4', 'm.02746s4', 'm.05pyj', 'm.02zpl5', 'm.017tgt'],
   []]],
 [[['percentage: 75.5, religion: Judaism']],
  [['end_date: 1983, start_date: 1973, organization: Chief Rabbinate of Israel, role: Chief Rabbi, leader: Ovadia Yosef']],
  [['Entering Heaven alive',
    'Omnipotence',
    'Unitarian monotheism',
    'Omniscience',
    'Tikkun olam',
    'Divine simplicity'],
   []]])

In [ ]:
facts = []
for i in paths:
    for j in paths[i]:
        fact = paths[i][j]['fact']
        facts.append(fact)
        paths[i].update({j: fact})
prompt = question_prompt.format("\n".join(facts), question)
response = run_llm(prompt, args.temperature, args.max_length, args.openai_api_key, args.llm)
output = {"question": question, "result": response, "paths": paths}

In [8]:
paths[topic_name]

{'people.deceased_person.place_of_burial': {'entities_id': ['m.0n5vjkf'],
  'entities_name': ['Sanhedria Cemetery'],
  'fact': 'Ovadia Yosef is buried in Sanhedria Cemetery.'},
 'people.person.nationality': {'entities_id': ['m.03spz'],
  'entities_name': ['Israel'],
  'fact': 'Ovadia Yosef is an Israeli national.'},
 'people.person.profession': {'entities_id': ['m.0djbw'],
  'entities_name': ['Rabbi'],
  'fact': 'Ovadia Yosef is a Rabbi by profession.'},
 'people.person.religion': {'entities_id': ['m.03_gx', 'm.0f2qf'],
  'entities_name': ['Judaism', 'Haredi Judaism'],
  'fact': 'Ovadia Yosef practices Judaism, specifically Haredi Judaism.'},
 'religion.religious_leader.religious_leadership': {'entities_id': ['m.048bcbz'],
  'entities_name': ['end_date: 1983, start_date: 1973, organization: Chief Rabbinate of Israel, role: Chief Rabbi, jurisdiction: Israel'],
  'fact': 'Ovadia Yosef served as the Chief Rabbi of Israel from 1973 to 1983.'},
 'people.person.nationality->location.statisti

In [ ]:
len(paths[topic_name])